In [1]:
import math
from matplotlib.pyplot import *
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from numpy.random import standard_normal
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import math

## Browmian motion class

In [2]:
class European_Call_Payoff:

    def __init__(self, strike):
        self.strike = strike

    def get_payoff(self, stock_price):
        if stock_price > self.strike:
            return stock_price - self.strike
        else:
            return 0


class GeometricBrownianMotion:

    def simulate_paths(self):
        while(self.T - self.dt > 0):
            dWt = np.random.normal(0, 1)  # Brownian motion
            dYt = 1 + self.drift*self.dt + self.volatility*math.sqrt(self.dt)*dWt  # Change in price
            self.current_price = dYt*self.current_price # Add the change to the current price
            self.prices.append(self.current_price)  # Append new price to series
            self.T -= self.dt  # Accound for the step in time 

    def __init__(self, initial_price, drift, volatility, dt, T):
        self.current_price = initial_price
        self.initial_price = initial_price
        self.drift = drift
        self.volatility = volatility
        self.dt = dt
        self.T = T
        self.prices = []
        self.simulate_paths()

## Heston pricer

In [3]:
class HestonPricer:

    def simulate_paths(self):
        while(self.T - self.dt > 0):
            
            # Normal distribution volatility
            epsilonV = np.random.normal(0, 1) 
            #compute volatility
            
             # Normal distribution for the price
            epsilonS = np.random.normal(0, 1)  
            
            self.current_price = self.current_price*(1 + self.mu*self.dt + 
                                                      math.sqrt(self.current_volatility*self.dt)*epsilonS)
            self.current_volatility= np.abs((self.current_volatility 
                                     +self.k*(self.teta-self.current_volatility)*self.dt 
                                     +self.volatility_std*(math.sqrt(self.current_volatility*self.dt))*epsilonV))
                    
            self.prices.append(self.current_price)  # Append new price to series
            self.T -= self.dt  # Accound for the step in time 

    def __init__(self, mu, teta, k, volatility_std, initial_price, initial_volatility, dt, T):
        
        #heston parameters 
        self.mu = mu
        self.teta = teta
        self.k = k
        self.volatility_std= volatility_std
        #initialization parameters
        self.current_price = initial_price
        self.initial_price = initial_price
        self.initial_volatility = initial_volatility
        self.current_volatility = initial_volatility
       
        self.dt = dt
        self.T = T
        self.prices = []
        self.simulate_paths()
        

## Monte carlo function

In [4]:
def monteCarlo(paths = [], spot_prices = [], interest_rates = [], volatilities = [], expiry_times = [],
               strike_prices = [], drift = 0.1, dt = 1/252, mu = 0.1, 
               teta = 0.1, k = 0.1, volatility_std = 0.01, flag=1):
    data= []
    np.random.seed(32)
    for sigma in tqdm(volatilities):

        for r in interest_rates:

            for T in expiry_times:

                for S0 in spot_prices:

                    for K in strike_prices:

                        # Monte Carlo simulation
                        price_paths=[] 

                        for i in range(0, paths):
                            if flag == 1:
                                price_paths.append(GeometricBrownianMotion(S0, drift, sigma, dt, T/252).prices)
                                # price = analytical_call_pricer(r, sigma, S0, T,K,option_type)
                            if flag == 2:
                                price_paths.append(HestonPricer(mu, teta, k, volatility_std, 
                                                                S0, sigma, dt, T/252).prices)

                        call_payoffs = []
                        ec = European_Call_Payoff(K)

                        for price_path in price_paths:
                            # We get the last stock price in the series generated by GBM to determin the payoff and discount it by one year
                            call_payoffs.append(ec.get_payoff(price_path[-1])/((1 + r)**(T/252)))  
                        #adding data to our dataset
                        data.append({
                            'volatility':sigma,
                            'interest_rate':r,
                            'expiry_time':T,
                            'initial_price':S0,
                            'strike_price':K,
                            'price':np.average(call_payoffs)
                        })

    df = pd.DataFrame.from_dict(data)
    df.head()
    X=df[['volatility','interest_rate','expiry_time','initial_price','strike_price']].values
    y= df[['price']].values
    X_train, X_val, Y_train, y_val= train_test_split(X, y, test_size=0.20)
    return X_train, X_val, Y_train, y_val



## Black Scholes

In [5]:
paths=10000
spot_prices = [v for v in range(5,100,20)]
interest_rates= [0.01, 0.015, 0.05, 0.043]
volatilities = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
expiry_times = [7, 14, 21, 28, 35, 42, 49]
strike_prices = [v for v in range(30,70,10)]
drift = .08
dt = 1/252

X_train, X_val, Y_train, y_val = monteCarlo(paths = paths, spot_prices = spot_prices, interest_rates = interest_rates,
                                            volatilities = volatilities, expiry_times = expiry_times, 
                                            strike_prices = strike_prices, drift = drift, dt= dt, flag = 1)

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:52<00:00, 18.72s/it]


In [6]:
print(X_train.shape, Y_train.shape)

print(X_val.shape, y_val.shape)

(2688, 5) (2688, 1)
(672, 5) (672, 1)


# Black Scholes - Neural network training

In [7]:
model = Sequential()
model.add(Dense(100, input_dim=5, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='relu'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy', 'mse', 'mae'])

In [9]:
model.fit(X_train, Y_train, epochs=200, batch_size=64, validation_data=(X_val,y_val))

Epoch 1/200
42/42 [==============================] - 4s 23ms/step - loss: 516.0301 - accuracy: 0.4295 - mse: 516.0301 - mae: 14.1773 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 2/200
42/42 [==============================] - 0s 8ms/step - loss: 481.1510 - accuracy: 0.4460 - mse: 481.1510 - mae: 13.2740 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 3/200
42/42 [==============================] - 1s 15ms/step - loss: 501.8728 - accuracy: 0.4449 - mse: 501.8728 - mae: 13.7433 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 4/200
42/42 [==============================] - 0s 12ms/step - loss: 470.8484 - accuracy: 0.4289 - mse: 470.8484 - mae: 13.2897 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 5/200
42/42 [==============================] - 1s 14ms/step - loss: 461.3775 - accuracy: 0.4471 - mse: 461.3775 - mae: 13.1364 - va

Epoch 39/200
42/42 [==============================] - 0s 10ms/step - loss: 501.5487 - accuracy: 0.4546 - mse: 501.5487 - mae: 13.6910 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 40/200
42/42 [==============================] - 0s 8ms/step - loss: 515.0174 - accuracy: 0.4436 - mse: 515.0174 - mae: 13.8981 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 41/200
42/42 [==============================] - 0s 10ms/step - loss: 478.7376 - accuracy: 0.4456 - mse: 478.7376 - mae: 13.4006 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 42/200
42/42 [==============================] - 0s 11ms/step - loss: 491.4707 - accuracy: 0.4616 - mse: 491.4707 - mae: 13.4004 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 43/200
42/42 [==============================] - 0s 10ms/step - loss: 487.4551 - accuracy: 0.4600 - mse: 487.4551 - mae: 13.2845

Epoch 77/200
42/42 [==============================] - 0s 10ms/step - loss: 484.3365 - accuracy: 0.4504 - mse: 484.3365 - mae: 13.3639 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 78/200
42/42 [==============================] - 0s 10ms/step - loss: 488.0854 - accuracy: 0.4548 - mse: 488.0854 - mae: 13.2987 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 79/200
42/42 [==============================] - 0s 10ms/step - loss: 449.4835 - accuracy: 0.4540 - mse: 449.4835 - mae: 12.7637 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 80/200
42/42 [==============================] - 0s 9ms/step - loss: 491.5038 - accuracy: 0.4612 - mse: 491.5038 - mae: 13.4319 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 81/200
42/42 [==============================] - 0s 8ms/step - loss: 468.9430 - accuracy: 0.4526 - mse: 468.9430 - mae: 13.0262 

Epoch 115/200
42/42 [==============================] - 0s 7ms/step - loss: 481.4882 - accuracy: 0.4581 - mse: 481.4882 - mae: 13.3480 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 116/200
42/42 [==============================] - 0s 7ms/step - loss: 469.2379 - accuracy: 0.4700 - mse: 469.2379 - mae: 12.9823 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 117/200
42/42 [==============================] - 0s 6ms/step - loss: 492.0246 - accuracy: 0.4423 - mse: 492.0246 - mae: 13.7097 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 118/200
42/42 [==============================] - 0s 6ms/step - loss: 478.7697 - accuracy: 0.4604 - mse: 478.7697 - mae: 13.1800 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 119/200
42/42 [==============================] - 0s 6ms/step - loss: 494.5660 - accuracy: 0.4401 - mse: 494.5660 - mae: 13.678

Epoch 153/200
42/42 [==============================] - 0s 8ms/step - loss: 496.8359 - accuracy: 0.4470 - mse: 496.8359 - mae: 13.6475 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 154/200
42/42 [==============================] - 0s 9ms/step - loss: 471.6618 - accuracy: 0.4597 - mse: 471.6618 - mae: 13.1248 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 155/200
42/42 [==============================] - 0s 7ms/step - loss: 470.7854 - accuracy: 0.4628 - mse: 470.7854 - mae: 13.0301 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 156/200
42/42 [==============================] - 0s 7ms/step - loss: 502.8195 - accuracy: 0.4499 - mse: 502.8195 - mae: 13.6883 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 157/200
42/42 [==============================] - 0s 7ms/step - loss: 450.7279 - accuracy: 0.4748 - mse: 450.7279 - mae: 12.610

42/42 [==============================] - 0s 9ms/step - loss: 462.8335 - accuracy: 0.4670 - mse: 462.8335 - mae: 12.8413 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 191/200
42/42 [==============================] - 0s 9ms/step - loss: 500.0308 - accuracy: 0.4571 - mse: 500.0308 - mae: 13.7270 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 192/200
42/42 [==============================] - 0s 7ms/step - loss: 478.2302 - accuracy: 0.4515 - mse: 478.2302 - mae: 13.2444 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 193/200
42/42 [==============================] - 0s 8ms/step - loss: 454.3096 - accuracy: 0.4686 - mse: 454.3096 - mae: 12.7192 - val_loss: 466.4959 - val_accuracy: 0.4405 - val_mse: 466.4959 - val_mae: 13.2376
Epoch 194/200
42/42 [==============================] - 0s 7ms/step - loss: 519.9239 - accuracy: 0.4434 - mse: 519.9239 - mae: 14.0788 - val_loss: 

In [10]:
# testing the neural network
# Generating a test dataset

nb_sample=1000
data= []
for n in tqdm(range(nb_sample)):
    sigma= np.random.uniform(low=volatilities[0],high=volatilities[-1])
    r = np.random.uniform(low=interest_rates[0],high=interest_rates[-1])
    T= np.random.uniform(low=expiry_times[0],high=expiry_times[-1])
    S0= np.random.uniform(low=spot_prices[0],high=spot_prices[-1])
    K= np.random.uniform(low=(1-0.2)*S0,high=(1+0.2)*S0)
    price_paths=[] 
    for i in range(0, paths):
        price_paths.append(GeometricBrownianMotion(S0, drift, sigma, dt, T/252).prices)
    #price = analytical_call_pricer(r, sigma, S0, T,K,option_type)
    call_payoffs = []
    ec = European_Call_Payoff(K)

    for price_path in price_paths:
        call_payoffs.append(ec.get_payoff(price_path[-1])/((1 + r)**(T/252)))  # We get the last stock price in the series generated by GBM to determin the payoff and discount it by one year 
    data.append({
                        'volatility':sigma,
                        'interest_rate':r,
                        'expiry_time':T,
                        'initial_price':S0,
                        'strike_price':K,
                        'price':np.average(call_payoffs)
                    })
df = pd.DataFrame.from_dict(data)
df.head()
X_test=df[['volatility','interest_rate','expiry_time','initial_price','strike_price']].values
y_test= df[['price']].values



100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:38<00:00, 26.22it/s]


In [11]:
loss, _, _, _ = model.evaluate(X_test, y_test)
print("test loss:", loss)

32/32 [==============================] - 0s 2ms/step - loss: 17.6800 - accuracy: 0.1320 - mse: 17.6800 - mae: 2.5117
test loss: 17.67999267578125


## Heston - Neural network training

In [12]:
mu = 0.1
teta = 0.04
k = 2
volatility_std = 0.3
X_train, X_val, Y_train, y_val = monteCarlo(paths = paths, spot_prices = spot_prices, interest_rates = interest_rates,
                                            volatilities = volatilities, expiry_times = expiry_times, 
                                            strike_prices = strike_prices, dt= dt, mu = mu, 
                                            teta = teta, k = k, volatility_std = volatility_std, flag=2)



100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [03:37<00:00, 36.27s/it]


In [13]:
model = Sequential()
model.add(Dense(100, input_dim=5, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='relu'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=200, batch_size=64, validation_data=(X_val,y_val))

Epoch 1/200
42/42 [==============================] - 2s 21ms/step - loss: 139.8159 - accuracy: 0.1522 - val_loss: 2.3629 - val_accuracy: 0.3676
Epoch 2/200
42/42 [==============================] - 0s 5ms/step - loss: 1.2842 - accuracy: 0.3873 - val_loss: 0.6753 - val_accuracy: 0.3676
Epoch 3/200
42/42 [==============================] - 0s 7ms/step - loss: 0.6737 - accuracy: 0.3783 - val_loss: 0.7561 - val_accuracy: 0.3676
Epoch 4/200
42/42 [==============================] - 0s 9ms/step - loss: 0.6195 - accuracy: 0.3937 - val_loss: 0.5765 - val_accuracy: 0.3676
Epoch 5/200
42/42 [==============================] - 0s 10ms/step - loss: 0.5495 - accuracy: 0.3820 - val_loss: 0.7014 - val_accuracy: 0.3676
Epoch 6/200
42/42 [==============================] - 0s 11ms/step - loss: 0.5587 - accuracy: 0.4018 - val_loss: 0.7440 - val_accuracy: 0.3676
Epoch 7/200
42/42 [==============================] - 1s 13ms/step - loss: 0.6020 - accuracy: 0.3655 - val_loss: 0.5676 - val_accuracy: 0.3676
Epoch 8

42/42 [==============================] - 0s 9ms/step - loss: 0.6763 - accuracy: 0.3710 - val_loss: 1.1178 - val_accuracy: 0.3676
Epoch 59/200
42/42 [==============================] - 0s 9ms/step - loss: 0.8302 - accuracy: 0.3677 - val_loss: 1.0662 - val_accuracy: 0.3676
Epoch 60/200
42/42 [==============================] - 0s 6ms/step - loss: 0.7462 - accuracy: 0.3922 - val_loss: 0.7774 - val_accuracy: 0.3676
Epoch 61/200
42/42 [==============================] - 0s 6ms/step - loss: 0.6073 - accuracy: 0.3752 - val_loss: 0.8672 - val_accuracy: 0.3676
Epoch 62/200
42/42 [==============================] - 0s 5ms/step - loss: 0.5605 - accuracy: 0.3799 - val_loss: 0.5548 - val_accuracy: 0.3676
Epoch 63/200
42/42 [==============================] - 0s 6ms/step - loss: 0.4881 - accuracy: 0.3792 - val_loss: 0.5457 - val_accuracy: 0.3676
Epoch 64/200
42/42 [==============================] - 0s 5ms/step - loss: 0.4441 - accuracy: 0.3806 - val_loss: 0.5642 - val_accuracy: 0.3676
Epoch 65/200
42/42 

Epoch 116/200
42/42 [==============================] - 0s 6ms/step - loss: 0.5102 - accuracy: 0.3913 - val_loss: 0.7108 - val_accuracy: 0.3676
Epoch 117/200
42/42 [==============================] - 0s 6ms/step - loss: 0.6538 - accuracy: 0.3804 - val_loss: 0.5206 - val_accuracy: 0.3676
Epoch 118/200
42/42 [==============================] - 0s 6ms/step - loss: 0.4338 - accuracy: 0.3840 - val_loss: 0.5530 - val_accuracy: 0.3676
Epoch 119/200
42/42 [==============================] - 0s 6ms/step - loss: 0.4887 - accuracy: 0.3802 - val_loss: 0.5661 - val_accuracy: 0.3676
Epoch 120/200
42/42 [==============================] - 0s 5ms/step - loss: 0.5112 - accuracy: 0.3908 - val_loss: 0.5417 - val_accuracy: 0.3676
Epoch 121/200
42/42 [==============================] - 0s 6ms/step - loss: 0.4979 - accuracy: 0.3746 - val_loss: 0.8188 - val_accuracy: 0.3661
Epoch 122/200
42/42 [==============================] - 0s 6ms/step - loss: 0.6246 - accuracy: 0.3756 - val_loss: 0.5834 - val_accuracy: 0.3676

Epoch 173/200
42/42 [==============================] - 0s 6ms/step - loss: 0.5020 - accuracy: 0.3713 - val_loss: 0.6309 - val_accuracy: 0.3676
Epoch 174/200
42/42 [==============================] - 0s 6ms/step - loss: 0.5281 - accuracy: 0.3841 - val_loss: 0.5921 - val_accuracy: 0.3676
Epoch 175/200
42/42 [==============================] - 0s 7ms/step - loss: 0.4796 - accuracy: 0.3889 - val_loss: 0.5610 - val_accuracy: 0.3676
Epoch 176/200
42/42 [==============================] - 0s 7ms/step - loss: 0.4494 - accuracy: 0.3924 - val_loss: 0.5197 - val_accuracy: 0.3676
Epoch 177/200
42/42 [==============================] - 0s 6ms/step - loss: 0.4318 - accuracy: 0.3781 - val_loss: 0.7131 - val_accuracy: 0.3676
Epoch 178/200
42/42 [==============================] - 0s 7ms/step - loss: 0.4851 - accuracy: 0.3816 - val_loss: 0.6370 - val_accuracy: 0.3676
Epoch 179/200
42/42 [==============================] - 0s 6ms/step - loss: 0.5718 - accuracy: 0.3839 - val_loss: 0.6201 - val_accuracy: 0.3676

In [14]:
# testing the neural network
# Generating a test dataset

nb_sample=1000
data= []
for n in tqdm(range(nb_sample)):
    sigma= np.random.uniform(low=volatilities[0],high=volatilities[-1])
    r = np.random.uniform(low=interest_rates[0],high=interest_rates[-1])
    T= np.random.uniform(low=expiry_times[0],high=expiry_times[-1])
    S0= np.random.uniform(low=spot_prices[0],high=spot_prices[-1])
    K= np.random.uniform(low=(1-0.2)*S0,high=(1+0.2)*S0)
    price_paths=[] 
    for i in range(0, paths):
        price_paths.append(HestonPricer(mu, teta, k, volatility_std, 
                                        S0, sigma, dt, T/252).prices)
    #price = analytical_call_pricer(r, sigma, S0, T,K,option_type)
    call_payoffs = []
    ec = European_Call_Payoff(K)

    for price_path in price_paths:
        call_payoffs.append(ec.get_payoff(price_path[-1])/((1 + r)**(T/252)))  # We get the last stock price in the series generated by GBM to determin the payoff and discount it by one year 
    data.append({
                        'volatility':sigma,
                        'interest_rate':r,
                        'expiry_time':T,
                        'initial_price':S0,
                        'strike_price':K,
                        'price':np.average(call_payoffs)
                    })
df = pd.DataFrame.from_dict(data)
df.head()
X_test=df[['volatility','interest_rate','expiry_time','initial_price','strike_price']].values
y_test= df[['price']].values


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:00<00:00, 16.43it/s]


In [16]:
loss, _ = model.evaluate(X_test, y_test)
print("test loss, test acc:", loss)

32/32 [==============================] - 0s 2ms/step - loss: 0.6178 - accuracy: 0.0090
test loss, test acc: 0.6178281903266907
